In [11]:
import csv
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn import model_selection
from sklearn import svm
import numpy as np
from scipy import stats
from collections import Counter
from sklearn.metrics import confusion_matrix
import sklearn.datasets as datasets
from sklearn.metrics import classification_report as report
from sklearn.datasets import make_classification
from numpy import argmax
from sklearn.svm import SVC
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

data="process.csv"

In [12]:
df = pd.read_csv(data)
df.head()


,Shape,formula_pretty,formula_anonymous,volume,density,density_atomic,energy_per_atom,band_gap,efermi,is_gap_direct,is_metal,is_magnetic,ordering,total_magnetization,theoretical
0,0,AlPbO3,ABC3,56.379268,8.311042,11.275854,-6.651740,0.000,6.457979,0,1,0,0,0.032580,1
1,2,Ba2CoMoO6,ABC2D6,277.167172,6.296940,13.858359,-7.604548,1.274,3.062452,1,0,1,2,0.000002,1
2,0,Ba2CoMoO6,ABC2D6,276.766068,6.306065,13.838303,-7.613764,0.000,3.727364,0,1,1,2,0.010982,0
3,4,Ba3HoIrRuO9,ABCD3E9,448.998994,7.501607,14.966633,-7.461055,0.000,3.778487,0,1,1,1,4.000589,1
4,4,Ba3LiOs2O9,AB2C3D9,428.639080,7.309243,14.287969,-7.343171,0.000,3.513875,0,1,1,1,6.011189,0


In [13]:
df = df.fillna(-2)

In [14]:
features=['volume', 'density', 'density_atomic', 'energy_per_atom','band_gap', 'efermi','is_gap_direct', 'is_metal', 'is_magnetic', 'ordering','total_magnetization','theoretical']  
X = df.loc[0:len(df), features].values
Y = df.loc[0:len(df), "Shape"].values

In [15]:
X

array([[5.63792677e+01, 8.31104232e+00, 1.12758535e+01, ...,
        0.00000000e+00, 3.25802000e-02, 1.00000000e+00],
       [2.77167172e+02, 6.29693952e+00, 1.38583586e+01, ...,
        2.00000000e+00, 2.10000000e-06, 1.00000000e+00],
       [2.76766068e+02, 6.30606538e+00, 1.38383034e+01, ...,
        2.00000000e+00, 1.09821000e-02, 0.00000000e+00],
       ...,
       [1.29163476e+02, 6.02477887e+00, 1.29163476e+01, ...,
        0.00000000e+00, 3.93313200e-01, 1.00000000e+00],
       [6.38148029e+01, 3.54957838e+00, 1.27629606e+01, ...,
        1.00000000e+00, 2.45797360e+00, 1.00000000e+00],
       [5.17705612e+01, 8.19502268e+00, 1.03541122e+01, ...,
        0.00000000e+00, 3.50000000e-06, 1.00000000e+00]])

OverSampling

In [16]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (X, Y, test_size=0.15, random_state=42)
print(len(X_train),len(X_test))
oversample = SMOTE()#oversampling of data
X_train, Y_train = oversample.fit_resample(X_train, Y_train)


12344 2179


In [17]:
print(len(X_train))

39991


In [18]:
def CRandCM(method,X_test,Y_test):#a function to print classification report and confusion matrix
    Y_pred=method.predict(X_test)
    cr = report(Y_test,  Y_pred)
    cm = confusion_matrix(Y_test, Y_pred)
    print(cr)
    print(cm)

SVM RBF

In [20]:
svc = SVC(kernel='rbf',C=19,gamma=0.05)
svc.fit(X_train,Y_train)


SVC(C=19, gamma=0.05)

In [21]:
accuracy = svc.score(X_test, Y_test)
print("Accuracy:", accuracy)
Y_pred=svc.predict(X_test)
print(report(Y_test, Y_pred))

Accuracy: 0.9150986691142726
              precision    recall  f1-score   support

           0       0.98      0.95      0.97      1000
           1       0.86      0.94      0.90       362
           2       0.77      0.81      0.79       148
           3       0.86      0.88      0.87       115
           4       0.92      0.87      0.89        39
           5       0.87      0.86      0.87       300
           6       0.91      0.87      0.89       215

    accuracy                           0.92      2179
   macro avg       0.88      0.88      0.88      2179
weighted avg       0.92      0.92      0.92      2179



In [22]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [23]:
c=[]
for i in range(19,40,5):
    c.append(i)

In [24]:
print(c)

[19, 24, 29, 34, 39]


In [25]:
param_grid = {'C': c, 'gamma': [0.01,0.05,0.1,0.2]}
svc = SVC(kernel='rbf')
grid_search = GridSearchCV(svc, param_grid, cv=5,verbose=1, n_jobs=-1)
grid_search.fit(X_train, Y_train)
print("Best hyperparameters: ", grid_search.best_params_)



Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
Y_pred = grid_search.best_estimator_.predict(X_test)
print(report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1000
           1       0.85      0.95      0.90       362
           2       0.80      0.79      0.80       148
           3       0.90      0.88      0.89       115
           4       0.94      0.85      0.89        39
           5       0.88      0.85      0.87       300
           6       0.90      0.87      0.89       215

    accuracy                           0.92      2179
   macro avg       0.90      0.88      0.89      2179
weighted avg       0.92      0.92      0.92      2179

